# Assignment 1

Cài đặt thư viện

In [9]:
%conda install numpy pandas

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


Import thư viện

In [10]:
import numpy as np
import pandas as pd

Các hàm hỗ trợ quá trình tính toán

In [11]:
ANSWER_COL = 'Answer'
IS_VALID_COL = 'IsValid'
SCORE_COL = 'Score'

def is_file_name_with_txt_extension(file_name):
    """ Kiểm tra xem tên file có đuôi .txt không
    
    Args:
        file_name (str): tên file
        
    Returns:
        bool: Return True nếu hàm có đuôi .txt, False nếu không
    """

    if file_name.endswith('.txt'):
        return True
    else:
        return False


def add_txt_extension(file_name):
    """ Thêm đuôi .txt vào tên file nếu chưa có, giữ nguyên nếu đã có đuôi .txt

    Args:
        file_name (str): tên file

    Returns:
        str: Tên file đã có đuôi .txt
    """

    if is_file_name_with_txt_extension(file_name):
        return file_name
    else:
        return file_name + '.txt'
    
def is_digit(string):
    """ Kiểm tra string truyền vào có phải là số không
    
    Args:
        string (str): string cần kiểm tra
        
    Returns:
        bool: Trả lại True nếu string là số, False nếu không
    """
    try:
        int(string)
        return True
    except ValueError:
        return False


def is_row_valid(row):
    """ Kiểm tra xem row truyền vào có hợp lệ hay không
        - Hợp lệ khi cả row có đủ 26 string sau khi đã tách bởi dấu phẩy, Mã số sinh viên bắt đầu bằng N và sau N là số.

    Args:
        row (str): Row cần kiểm tra

    Returns:
        bool: True nếu row là hợp lệ, False nếu không
    """
    data = row[ANSWER_COL].split(',')
    if len(data) != 26:
        print('Invalid line of data: does not contain exactly 26 values:')
        print(row[ANSWER_COL] + '\n')
        return False
    student_id = data[0]
    if len(student_id) != 9 or student_id[0] != 'N' or (not is_digit(student_id[1:])):
        print('Invalid line of data: N# is invalid')
        print(row[ANSWER_COL] + '\n')
        return False
    return True 

def grade_the_exams(row):
    """ Tính điểm cho mỗi dữ liệu hợp lệ.

    Args:
        student_info (str): dữ liệu của 1 row tương ứng với 1 sinh viên.

    Returns:
        str: Mã sinh viên
        int: điểm của sinh viên
    """

    correct_answer_key = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
    correct_answer_array = correct_answer_key.split(',')
    data = row[ANSWER_COL].split(',')
    student_answer = data[1:]
    point = 0
    for index, answer in enumerate(student_answer):
        if len(answer.strip()) == 0:
            continue
        if answer.strip() == correct_answer_array[index]:
            point += 4
        else:
            point -= 1
    return data[0], point


Mở file với Pandas.

- Do toàn bộ dữ liệu là trên mỗi dòng là khác nhau (bao gồm cả hợp lệ và không hợp lệ) nên việc đưa toàn bộ dữ liệu vào DataFrame ngăn cách bởi dấu phẩy trở lên khó khăn, số cột của DataFrame phụ thuộc vào line đầu tiên. Vì vậy cách giải quyết là đưa toàn bộ dữ liệu vào 1 dòng và xử lý sau

In [12]:
# Nhận input (tên file) từ người dùng.
input_file_name = input('Enter a class file to grade (i.e. class1 for class1.txt): ')
# Kiểm tra tên file có đuôi .txt không, nếu có thì giữ nguyên, nếu không thì cần thêm đuôi .txt 
valid_file_name = add_txt_extension(input_file_name)
try:
    # Đưa dữ liệu vào DataFrame, do cả dòng không có ký tự space nào, nên có thể sử dụng sep=' ', giúp đưa cả dòng vào 1 cột.
    df = pd.read_csv(valid_file_name, sep=' ' ,header=None) 
    print('\n Successfully opened ' + valid_file_name + '\n')
except FileNotFoundError:
    print('\n File cannot be found. \n')
    exit()

Enter a class file to grade (i.e. class1 for class1.txt): class2

 Successfully opened class2.txt



Làm sạch sữ liệu, thêm 1 cột tên là 'IsValid' để đánh dấu những dòng nào là dòng chứa dữ liệu hợp lệ, dòng nào không.

In [13]:
print('**** ANALYZING **** \n')
# Đổi tên cột hiện tại thành cột tên "Answer", giúp cột có ý nghĩa hơn
df.columns = [ANSWER_COL] 
# Thêm cột mới chứa thông tin về dòng đó có dữ liệu hợp lệ hay không.
# Vì hàm is_row_valid(row) trả lại giá trị True/False nên cột này chứa giá trị True/False
df[IS_VALID_COL] = df.apply(lambda row: is_row_valid(row), axis=1)
# Chuyển các dữ liệu hợp lệ sang một DataFrame mới tiện cho việc tính toán
df_valid = df[df[IS_VALID_COL]]

# Dựa vào shape của DataFrame để đếm số lượng dữ liệu hợp lệ trong DataFrame(df_valid)
valid_line = df_valid.shape[0]
# Số lượng invalid bằng tổng trừ đi số lượng valid
invalid_line = df.shape[0] - valid_line

if invalid_line == 0:
    print('No errors found!\n')


**** ANALYZING **** 

Invalid line of data: does not contain exactly 26 values:
N00000023,,A,D,D,C,B,D,A,C,C,,C,,B,A,C,B,D,A,C,A,A

Invalid line of data: N# is invalid
N0000002,B,A,D,D,C,B,D,A,C,D,D,D,A,,A,C,D,,A,C,A,A,B,D,D

Invalid line of data: N# is invalid
NA0000027,B,A,D,D,,B,,A,C,B,D,B,A,,A,C,B,D,A,,A,A,B,D,D

Invalid line of data: does not contain exactly 26 values:
N00000035,B,A,D,D,B,B,,A,C,,D,B,A,B,A,A,B,D,A,C,A,C,B,D,D,A,A



Dựa vào DataFrame chứa những dữ liệu hợp lệ (df_valid) để tính toán điểm, đưa dữ liệu điểm vào một DataFrame mới (df_score) thuận tiện cho việc tính toán các giá trị statistic cũng như ghi kết qủa ra file

In [14]:
# Tính toán điểm dựa vào hàm grade_the_exams(row), đưa kết quả vào một DataFrame mới. 
df_score = df_valid.apply(lambda row: grade_the_exams(row), axis=1, result_type='expand') 

# Làm đẹp dữ liệu trong df_score

# Chuyển đổi cột chứa Mã số sinh viên thành index của DataFrame. 
# Trong bài tập lần này thì việc này thực sự không cần thiết lắm, 
# nhưng nếu có yêu cầu đưa ra nhanh điểm của một sinh viên thì việc này rất hữu dụng và tiện lợi.
df_score.set_index(0, inplace=True)

# Tên cho cột index là không cần thiết.
df_score.index.name = None

# Đổi tên cột thành score giúp DataFrame dễ hình dung.
df_score.rename(columns={1: SCORE_COL}, inplace=True)
df_score.head()

,Score
N00000021,68
N00000022,76
N00000024,73
N00000026,72
N00000028,73


Tính toán các đại lượng statistic dựa vào DataFrame Score (df_score) và in ra kết quả

In [15]:
mean_score = df_score[SCORE_COL].mean()
highest_score = df_score[SCORE_COL].max()
lowest_score = df_score[SCORE_COL].min()
range_score = highest_score - lowest_score
median_score = df_score[SCORE_COL].median()
print('**** REPORT **** \n')
print('Total valid lines of data: ' + str(valid_line))
print('Total invalid lines of data: ' + str(invalid_line))
print('Mean (average) score: ' + str(mean_score))
print('Highest score: ' + str(highest_score))
print('Lowest score: ' + str(lowest_score))
print('Range of scores: ' + str(range_score))
print('Median score: ' + str(median_score))


**** REPORT **** 

Total valid lines of data: 21
Total invalid lines of data: 4
Mean (average) score: 78.0
Highest score: 100
Lowest score: 66
Range of scores: 34
Median score: 76.0


In dữ liệu điểm số của các học sinh hợp lệ ra file dựa vào DataFrame Score (df_score)

In [16]:
# Lấy tên file không bao gồm đuôi .txt, thêm hậu tố _grades.txt tạo ra tên file cần ghi.
new_file_name = input_file_name.split('.')[0] + '_grades.txt'
df_score.to_csv(new_file_name, sep=',', header=False)